In [ ]:
import sys                                          # Read system parameters.
import os                                           # Interact with the operating system.
import numpy as np                                  # Work with multi-dimensional arrays and matrices.
import pandas as pd                                 # Manipulate and analyze data.
import scipy as sp                                   # Perform scientific computing and advanced mathematics.
import sklearn                                      # Perform data mining and analysis.
import re

In [ ]:
df = pd.read_excel("penalty_data.xlsx")

In [ ]:
print(df)

   IMAGE_NAME              ID_PENALTY REGISTRATION                  DATE_TIME  \
0       1.jpg              3723572555     GE212796      le 27/10/2023 à 20h51   
1      10.jpg         123146806 198 8    GE 278844        1 mars 2023 à 21:12   
2      11.jpg         240032607 066 7    GE 305362   Sa-13.01.2024  15:46 Uhr   
3      12.jpg              3720560535     GE212796     le 26/10/2023 à 20h01    
4      13.jpg                 3107808     GE206608   Montag 15.01.2024 12:41    
..        ...                     ...          ...                        ...   
82     84.jpg              3729239695     GE303735         25/02/2024 à 10h45   
83     85.jpg  VStV/923301724020/2023    GE-315431    04.03.2023 Um 19:27 Uhr   
84     86.jpg         002300801 007 2    GE 278844        4.12.2023 - 14:14 h   
85     87.jpg               231464006    GE 305998  10.07.2023 alle ore 12:09   
86      9.jpg              3736935725     GE278844      le 18/12/2023 à 20h39   

                           

##Date Column preparation

Split date and time into two separate columns

In [ ]:
# Define regex patterns to match different date and time formats
date_patterns = [
    r"\b(\d{1,2}[/]\d{1,2}[/]\d{4})\b",  # Matches date in format dd/mm/yyyy or d/m/yyyy
    r"\b(\d{1,2}\s(?:janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\s\d{4})\b",  # Matches date with month names
    r"\b(\d{1,2}\s(?:gennaio|febbraio|marzo|zprile|maggio|giugno|luglio|agosto|settembre|ottobre|novembre|dicembre)\s\d{4})\b",
    r"\b(\d{1,2}\s(?:januar|februar|märz|april|mai|juni|juli|august|september|oktober|november|dezember)\s\d{4})\b",
    r"\b(\d{1,2}\s(?:january|february|march|april|may|june|july|august|september|october|november|december)\s\d{4})\b",
    r"\b(\d{1,2}[. ]\s(?:janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\s\d{4})\b",
    r"\b(\d{1,2}[. ]\s(?:january|february|march|april|may|june|july|august|september|october|november|december)\s\d{4})\b",
    r"\b(\d{1,2}[.]\d{1,2}[.]\d{4})\b",  # Matches date in format dd.mm.yyyy or d.m.yyyy
    r"\b(\d{1,2}[-]\d{1,2}[-]\d{4})\b",  # Matches date in format dd-mm-yyyy or d-m-yyyy
    r"\b(\d{1,2}\s(?:jan\.|fév\.|mar\.|avr\.|mai|juin|juil\.|août|sept\.|oct\.|nov\.|déc\.)\s\d{4})\b"  # Matches date with abbreviated month names
]

time_patterns = [
    r"\b(\d{1,2}[:]\d{2})\b",  # Matches time in format hh:mm
    r"\b(\d{1,2}[h]\d{2})\b",  # Matches time in format hhhmm
    r"\b(\d{1,2}[:]\d{2}\s(?:Uhr)?)\b",  # Matches time with optional "Uhr" suffix
]

In [ ]:
# Extract date and time the DATE_TIME column and create new columns for date and time
def extract_date_time(row):
    date = None
    time = None
    for pattern in date_patterns:
        match = re.search(pattern, row, flags=re.IGNORECASE)
        if match:
            date = match.group(1)
            break
    for pattern in time_patterns:
        match = re.search(pattern, row, flags=re.IGNORECASE)
        if match:
            time = match.group(1)
            break
    return date, time

# Apply the function to each row
df['Date Penalty'] = df['DATE_TIME'].apply(lambda x: extract_date_time(x)[0])
df['Time Penalty'] = df['DATE_TIME'].apply(lambda x: extract_date_time(x)[1])

df.drop(columns=['DATE_TIME'], inplace=True)

In [ ]:
# Drop rows where the 'IMAGE_NAME' column matches any of the specified values
df = df[~df['IMAGE_NAME'].isin(['40.jpg', '80.jpg', '6.jpg'])]

In [ ]:
print(df)

   IMAGE_NAME              ID_PENALTY REGISTRATION  \
0       1.jpg              3723572555     GE212796   
1      10.jpg         123146806 198 8    GE 278844   
2      11.jpg         240032607 066 7    GE 305362   
3      12.jpg              3720560535     GE212796   
4      13.jpg                 3107808     GE206608   
..        ...                     ...          ...   
82     84.jpg              3729239695     GE303735   
83     85.jpg  VStV/923301724020/2023    GE-315431   
84     86.jpg         002300801 007 2    GE 278844   
85     87.jpg               231464006    GE 305998   
86      9.jpg              3736935725     GE278844   

                                             LOCATION  \
0   CHAMONIX MONT BLANC vers MACON ETREMBIERES - 7...   
1         rte. de Lausanne Vengeron, direction Genève   
2              Zürich 1, Walchestrasse / Neumühlequai   
3   ANNECY vers GENEVE . ST JULIEN EN GENEVOIS - 7...   
4                 Davos Dorf Bahnhofstrasse Höhe Coop   
..       

Unify the date format with the car_sales_data

In [ ]:
#desired_format = r"\b(\d{1,2}[/]\d{1,2}[/]\d{4})\b"

# Convert date to the desired format
def convert_to_desired_format(date_str):
    for pattern in date_patterns:
        match = re.search(pattern, date_str, flags=re.IGNORECASE)
        if match:
            # Extract day, month, and year components
            components = re.findall(r'\d+|\b(?:janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre|gennaio|febbraio|marzo|aprile|maggio|giugno|luglio|agosto|settembre|ottobre|novembre|dicembre|januar|februar|märz|april|mai|juni|juli|august|september|oktober|november|dezember|january|february|march|april|may|june|july|august|september|october|november|december)\b|\d{4,}', match.group(), flags=re.IGNORECASE)
            day = components[0]
            month = components[1]
            year = components[2]

            day = day.zfill(2)

            new_date_str = f"{day}/{month}/{year}"
            return new_date_str
    return date_str

df['Date Penalty'] = df['Date Penalty'].apply(convert_to_desired_format)


In [ ]:
#Convert month from a word to digits
def convert_date(date_str):
    month_mapping = {
    'janvier': '01', 'février': '02', 'mars': '03', 'avril': '04', 'mai': '05', 'juin': '06',
    'juillet': '07', 'août': '08', 'septembre': '09', 'octobre': '10', 'novembre': '11', 'décembre': '12',
    'january': '01', 'february': '02', 'march': '03', 'april': '04', 'may': '05', 'june': '06',
    'july': '07', 'august': '08', 'september': '09', 'october': '10', 'november': '11', 'december': '12',
    'gennaio': '01', 'febbraio': '02', 'marzo': '03', 'aprile': '04', 'maggio': '05', 'giugno': '06',
    'luglio': '07', 'agosto': '08', 'settembre': '09', 'ottobre': '10', 'novembre': '11', 'dicembre': '12',
    'januar': '01', 'februar': '02', 'märz': '03', 'april': '04', 'mai': '05', 'juni': '06',
    'juli': '07', 'august': '08', 'september': '09', 'oktober': '10', 'november': '11', 'dezember': '12'
}

    # Split the date string into day, month, and year
    parts = date_str.split('/')

    if parts[1].isdigit():
        month_numeric = parts[1]
    else:
        month_numeric = month_mapping[parts[1].lower()]

    new_date_str = f"{parts[0]}/{month_numeric}/{parts[2]}"

    return new_date_str

df['Date Penalty'] = df['Date Penalty'].apply(convert_date)

##Time Column preparation

In [ ]:
from datetime import datetime

def convert_to_consistent_time_format(time_str):
    try:
        # Parse the time string using different formats
        time_formats = ["%H:%M", "%Hh%M", "%H:%M:%S"]
        time_obj = None
        for fmt in time_formats:
            try:
                time_obj = datetime.strptime(time_str, fmt)
                break
            except ValueError:
                continue

        # If time_obj is None, it means the time_str couldn't be parsed
        if time_obj is None:
            return None

        # Convert the time to the standard format
        standard_time = datetime.strptime("17:35", "%H:%M")
        converted_time = standard_time.replace(hour=time_obj.hour, minute=time_obj.minute)

        return converted_time.strftime("%H:%M")

    except Exception as e:
        print("Error:", e)
        return None


df['Time Penalty'] = df['Time Penalty'].apply(convert_to_consistent_time_format)

##Plate Number Column preparation

Change column name

In [ ]:
pd.set_option('display.max_columns', None)
df.rename(columns = {'REGISTRATION': 'Plate Number'},inplace=True)

Unify the license plate format

In [ ]:
def extract_license_plate(text):
    # Define a regular expression pattern to match license plate numbers
    pattern = r'\b([A-Za-z]+)\W*(\d+)\b'

    # Find all matches in the text
    matches = re.findall(pattern, text)

    # Filter out matches that don't conform to the license plate number format
    valid_matches = [(letters, digits) for letters, digits in matches if len(letters) > 0 and len(digits) > 0]

    # Format valid matches with a space between letters and digits
    if valid_matches:
        return ' '.join(valid_matches[0])
    else:
        return ''

df['Plate Number'] = df['Plate Number'].apply(extract_license_plate)

In [ ]:
df.to_excel('penalty_data.xlsx', index=False)